In [ ]:
import torch
from datasets import load_dataset
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b0")
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")

inputs = preprocessor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print('x' + model.config.id2label[predicted_label]),

xtabby, tabby cat


(None,)

In [79]:
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
from time import perf_counter
import numpy as np


# For TPU acceleration
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
model_name = 'google/efficientnet-b7'

    
# Processor:
# - resize to the expected size
# - convert the image to a tensor [C, H, W] (channels × height × width)
# - normalize the pixels (often between -1 and 1 or 0 and 1)
# - add the batch dimension: [batch, C, H, W] e.g. how many samples (images) are fed to the model
processor = AutoImageProcessor.from_pretrained(model_name)
# Model: transform the tensor to embeddings or prediction
model = AutoModel.from_pretrained(model_name).to(device)

def embed(img_path):
    img = Image.open(img_path).convert('L').convert('RGB')
    # start = perf_counter()
    inputs = processor(images=img, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # stop = perf_counter()
    return outputs.pooler_output[0].numpy()

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

mona0 = embed('../data/base/mona-lisa-3-crop.jpeg')
mona1 = embed('../data/mona-lisa-1.jpg')

print("cosine_similarity: ", cosine_similarity(mona0, mona1))

cosine_similarity:  0.6427465


In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch
import numpy as np

# --- Embedding minimal ---
processor = AutoImageProcessor.from_pretrained("google/efficientnet-b0")
model = AutoModel.from_pretrained("google/efficientnet-b0")

def embed(img_path):
    img = Image.open(img_path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output[0].numpy()

# --- Similarité cosinus ---
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# --- Exemple ---
v1 = embed("image1.jpg")
v2 = embed("image2.jpg")
print("Similarité :", cosine_similarity(v1, v2))